<a href="https://colab.research.google.com/github/hellocybernetics/Tensorflow-Probability-Tutorials/blob/master/glm_fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip uninstall tensorflow -y
# !pip install --upgrade tensorflow==1.11
!pip install tensorflow-probability

In [0]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
sns.set_style()

tfd = tfp.distributions
tfe = tf.contrib.eager

In [0]:
tf.enable_eager_execution()

In [0]:
def make_dataset(n, d, link, scale=1., dtype=np.float32):
    model_coefficients = tfd.Uniform(
        low=np.array(-1, dtype),
        high=np.array(1, dtype)).sample(d, seed=42)
    
    radius = np.sqrt(2.)
    
    model_coefficients *= radius / tf.linalg.norm(model_coefficients)
    model_matrix = tfd.Normal(
        loc=np.array(0, dtype),
        scale=np.array(1, dtype)).sample([n, d], seed=43)
    
    scale = tf.convert_to_tensor(scale, dtype)
    linear_response = tf.tensordot(
        model_matrix, model_coefficients, axes=[[1], [0]])
    
    if link == 'linear':
        response = tfd.Normal(loc=linear_response, scale=scale).sample(seed=44)
    
    elif link == 'probit':
        response = tf.cast(
            tfd.Normal(loc=linear_response, scale=scale).sample(seed=44) > 0,
            dtype)
    
    elif link == 'logit':
        response = tfd.Bernoulli(logits=linear_response).sample(seed=44)
    
    else:
        raise ValueError('unrecognized true link: {}'.format(link))
    return model_matrix, response, model_coefficients

In [0]:
X, Y, w_true = make_dataset(n=int(500), d=3, link='probit')

X_train = X[:400]
X_test = X[400:]
Y_train = Y[:400]
Y_test = Y[400:]


model_probit = tfp.glm.BernoulliNormalCDF()

w, linear_response, is_converged, num_iter = tfp.glm.fit(
    model_matrix=X_train,
    response=Y_train,
    model=model_probit)

In [7]:
print("w_true: ", w_true.numpy())
print("w_fit: ", w.numpy())

w_true:  [1.125005  0.4412938 0.7345909]
w_fit:  [1.3105882  0.56357825 0.76087064]


In [0]:
Y = model_probit(tf.matmul(X_test, w[:, tf.newaxis]))

In [16]:
Y

(<tf.Tensor: id=1334, shape=(100, 1), dtype=float32, numpy=
 array([[0.52370226],
        [0.5038208 ],
        [0.29711306],
        [0.42350078],
        [0.54657376],
        [0.2701589 ],
        [0.46616298],
        [0.42828315],
        [0.44069675],
        [0.6410709 ],
        [0.34107396],
        [0.5321541 ],
        [0.53840053],
        [0.58155054],
        [0.30718696],
        [0.47295618],
        [0.429927  ],
        [0.22622141],
        [0.49543732],
        [0.7838849 ],
        [0.6293001 ],
        [0.44523463],
        [0.2882873 ],
        [0.4551715 ],
        [0.5571623 ],
        [0.65678406],
        [0.27041918],
        [0.34454897],
        [0.55384195],
        [0.85787845],
        [0.31325737],
        [0.65032303],
        [0.39172864],
        [0.3534933 ],
        [0.4130386 ],
        [0.65394634],
        [0.34674087],
        [0.51322377],
        [0.60290617],
        [0.491878  ],
        [0.4517678 ],
        [0.20064196],
        [0.20369

w_true:  [1.125005  0.4412938 0.7345909]
w_fit:  [0.30323732 0.15513557 0.17084502]


<tf.Tensor: id=158, shape=(1000000,), dtype=float32, numpy=array([1., 0., 1., ..., 0., 1., 1.], dtype=float32)>